# Task 3 — A/B Hypothesis Testing for Insurance Risk Analysis
### Week 3 — 10 Academy: Insurance Risk Analytics & Predictive Modeling

In [10]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind
from statsmodels.stats.proportion import proportions_ztest
import warnings
warnings.filterwarnings("ignore")

📌 2. Load Dataset

In [12]:
df = pd.read_csv("../data/insurance_data.csv", low_memory=False)
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827.0,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827.0,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827.0,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827.0,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827.0,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


📌 3. Create Required Metrics

In [13]:
# Claim Frequency
df["HasClaim"] = (df["TotalClaims"] > 0).astype(int)

# Margin
df["Margin"] = df["TotalPremium"] - df["TotalClaims"]

# Loss Ratio
df["LossRatio"] = np.where(df["TotalPremium"] > 0,
                           df["TotalClaims"] / df["TotalPremium"],
                           np.nan)

df[["HasClaim","Margin","LossRatio"]].head()


,HasClaim,Margin,LossRatio
0,0,21.929825,0.0
1,0,21.929825,0.0
2,0,0.000000,NaN
3,0,512.848070,0.0
4,0,0.000000,NaN


📌 4. Basic Portfolio Overview

In [14]:
print("Data Shape:", df.shape)

overall_freq = df["HasClaim"].mean()
overall_severity = df.loc[df["HasClaim"]==1, "TotalClaims"].mean()
overall_margin = df["Margin"].mean()

print("Overall Claim Frequency:", overall_freq)
print("Overall Severity:", overall_severity)
print("Overall Margin:", overall_margin)


Data Shape: (1000098, 55)
Overall Claim Frequency: 0.0027397315063123814
Overall Severity: 23305.40233064413
Overall Margin: -3.8462903968208604


📌 5. Test 1 — Provincial Risk Difference (Chi-Square Test)
### Hypothesis
- **H₀:** There are no risk differences across provinces  
- **H₁:** At least one province has significantly different claim frequency  

We use a **Chi-Square test** on the contingency table Province × HasClaim.

In [15]:
table = pd.crosstab(df["Province"], df["HasClaim"])
chi2, p, dof, expected = chi2_contingency(table)

print("Chi2:", chi2)
print("p-value:", p)

if p < 0.05:
    print("💡 Reject H0 — Provinces have significant risk differences")
else:
    print("✔️ Fail to reject H0 — No significant difference between provinces")


Chi2: 97.87315387252006
p-value: 1.1607771770235118e-17
💡 Reject H0 — Provinces have significant risk differences


📌 6. Provincial Summary Table (Optional Visualization)

In [16]:
prov_summary = df.groupby("Province").agg(
    policies=("HasClaim","count"),
    claim_freq=("HasClaim","mean"),
    loss_ratio=("LossRatio","mean")
).sort_values("claim_freq", ascending=False)

prov_summary

,policies,claim_freq,loss_ratio
Province,,,
Gauteng,393274,0.003362,0.429660
KwaZulu-Natal,169769,0.002845,0.265357
Limpopo,24836,0.002698,0.348712
North West,143263,0.002436,0.285393
Mpumalanga,52718,0.002428,0.392698
Western Cape,167906,0.002180,0.343317
Free State,8099,0.001358,0.106209
Northern Cape,6380,0.001254,0.203831
Eastern Cape,7137,0.000841,0.101468


📌 7. Test 2 — Claim Frequency Difference Between Two Zip Codes
We test the two most common zip codes using a **two-proportion z-test**.

### Hypothesis
- **H₀:** Claim frequencies between the two zip codes are equal  
- **H₁:** They differ  


In [17]:
# Two most common zip codes
zip_counts = df["PostalCode"].value_counts()
zipA = zip_counts.index[0]
zipB = zip_counts.index[1]

a = df[df["PostalCode"] == zipA]["HasClaim"]
b = df[df["PostalCode"] == zipB]["HasClaim"]

count = np.array([a.sum(), b.sum()])
nobs = np.array([len(a), len(b)])

zstat, p_zip = proportions_ztest(count, nobs)

print("Z-statistic:", zstat)
print("p-value:", p_zip)

if p_zip < 0.05:
    print(f"💡 Reject H0 — {zipA} and {zipB} have different claim frequencies")
else:
    print(f"✔️ Fail to reject H0 — No significant difference between {zipA} and {zipB}")


Z-statistic: -1.9162277114358377
p-value: 0.055336117551680726
✔️ Fail to reject H0 — No significant difference between 2000.0 and 122.0


📌 8. Test 3 — Margin Difference Between Zip Codes (t-test)
### Hypothesis
- **H₀:** Mean margin is equal between the two zip codes  
- **H₁:** The margins differ  

In [18]:
marginA = df[df["PostalCode"] == zipA]["Margin"].dropna()
marginB = df[df["PostalCode"] == zipB]["Margin"].dropna()

tstat, p_margin = ttest_ind(marginA, marginB, equal_var=False)

print("T-statistic:", tstat)
print("p-value:", p_margin)

if p_margin < 0.05:
    print(f"💡 Reject H0 — Margin differs between {zipA} and {zipB}")
else:
    print(f"✔️ Fail to reject H0 — No margin difference between {zipA} and {zipB}")


T-statistic: 1.1457170101298093
p-value: 0.25191600620729687
✔️ Fail to reject H0 — No margin difference between 2000.0 and 122.0


📌 9. Test 4 — Gender Risk Difference (Chi-Square Test)
### Hypothesis
- **H₀:** There is no risk difference between Women and Men  
- **H₁:** Claim frequency differs by gender  


In [19]:
gender_table = pd.crosstab(df["Gender"], df["HasClaim"])
chi2, p_gender, dof, exp = chi2_contingency(gender_table)

print("Chi2:", chi2)
print("p-value:", p_gender)

if p_gender < 0.05:
    print("💡 Reject H0 — Gender-based risk differences exist")
else:
    print("✔️ Fail to reject H0 — No gender-based differences")


Chi2: 7.850671752491598
p-value: 0.019735507070135053
💡 Reject H0 — Gender-based risk differences exist


📌 10. Final Interpretation Summary

In [20]:
print("===== SUMMARY =====")

print(f"Province test p={p:.6f} → {'Reject H0' if p<0.05 else 'Fail to reject H0'}")
print(f"Zip frequency test p={p_zip:.6f} → {'Reject H0' if p_zip<0.05 else 'Fail'}")
print(f"Zip margin test p={p_margin:.6f} → {'Reject H0' if p_margin<0.05 else 'Fail'}")
print(f"Gender test p={p_gender:.6f} → {'Reject H0' if p_gender<0.05 else 'Fail'}")


===== SUMMARY =====
Province test p=0.000000 → Reject H0
Zip frequency test p=0.055336 → Fail
Zip margin test p=0.251916 → Fail
Gender test p=0.019736 → Reject H0
